# Imports

In [19]:
# -*- coding: UTF-8 -*-
# Local packages:
import argparse
import logging
import os
import pickle
import random
import shutil
import time
from typing import Dict, Union

# 3rd party packages:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from tqdm import tqdm,trange
from tkinter import Tk
from tkinter.filedialog import askopenfilenames

#from torch.utils.tensorboard import SummaryWriter
import neptune
run = neptune.init_run(
    project="NTLAB/HypnoGAN",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhNGRjNDgzOC04OTk5LTQ0YTktYjQ4Ny1hMTE4NzRjNjBiM2EifQ==",
)


# TODO: Implement Neptune logger

# personal packages:
#from Data.preprocess import preprocess_data
#from model.timegan import TimeGAN
#from model.utils import timegan_trainer, timegan_generator

https://app.neptune.ai/NTLAB/HypnoGAN/e/HYPNOG-8


# Model

## Dataset class

In [20]:
class TimeGAN_Dataset(torch.utils.data.Dataset):
    """A time series dataset for TimeGAN.
    Args:
        data(numpy.ndarray): the padded dataset to be fitted. Has to transform to ndarray from DataFrame during initialize
        time(numpy.ndarray): the length of each data
    Parameters:
        - x (torch.FloatTensor): the real value features of the data
        - t (torch.LongTensor): the temporal feature of the data
    """
    def __init__(self,data, time=None):
        #sanity check data and time
        
        
        
        if isinstance(time,type(None)):
            time = [len(x) for x in data]
            
        if len(data) != len(time):
            raise ValueError( f"len(data) `{len(data)}` != len(time) {len(time)}")
        
        self.X = torch.FloatTensor(data)
        self.T = torch.LongTensor(time)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self,idx):
        return self.X[idx],self.T[idx]
    
    def collate_fn(self, batch):
        """Minibatch sampling
        """
        # Pad sequences to max length
        X_mb = [X for X in batch[0]]
        
        # The actual length of each data
        T_mb = [T for T in batch[1]]
        
        return X_mb, T_mb

## TimeGAN

In [21]:
class EmbeddingNetwork(nn.Module):
    """
    The embedding network (encoder) that maps the input data to a latent space.
    """
    def __init__(self, feature_dim, hidden_dim, num_layers, padding_value=0, max_seq_len=1000):
        super(EmbeddingNetwork, self).__init__()
        self.feature_dim = feature_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.padding_value = padding_value
        self. max_seq_len = max_seq_len

        #Embedder Architecture
        self.emb_rnn = nn.GRU(
            input_size=self.feature_dim,
            hidden_size=self.hidden_dim,
            num_layers=self.num_layers,
            batch_first=True,
        )
        self.emb_linear = nn.Linear(self.hidden_dim, self.hidden_dim)
        self.emb_sigmoid = nn.Sigmoid()

        
        # Init weights
        # Default weights of TensorFlow is Xavier Uniform for W and 1 or 0 for b
        # Reference: 
        # - https://www.tensorflow.org/api_docs/python/tf/compat/v1/get_variable
        # - https://github.com/tensorflow/tensorflow/blob/v2.3.1/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py#L484-L61

        with torch.no_grad():
            for name, param in self.emb_rnn.named_parameters():
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(param.data)
                elif 'weight_hh' in name:
                    nn.init.orthogonal_(param.data)
                elif 'bias_ih' in name:
                    param.data.fill_(1)
                elif 'bias_hh' in name:
                    param.data.fill_(1)

            for name, param in self.emb_linear.named_parameters():
                if 'weight' in name:
                    nn.init.xavier_uniform_(param)
                elif 'bias' in name:
                    param.data.fill_(0)
    def forward(self,X,T):
        """Forward pass of the embedding features from original space to latent space.
        Args:
            X: Input time series feature (B x S x F)
            T: INput temporal information (B)
        Returns:
            H: latent space embeddings (B x S x H)
        """
        # Dynamic RNN input for ignoring paddings

        X_pack = nn.utils.rnn.pack_padded_sequence(
            input =X,
            lengths=T,
            batch_first=True,
            enforce_sorted=False,
        )

        # 128*100*71
        H_o,H_t = self.emb_rnn(X_pack)

        #pad RNN output back to sequence length

        H_o,T = nn.utils.rnn.pad_packed_sequence(
            sequence=H_o,
            batch_first=True,
            padding_value=self.padding_value,
            total_length=self.max_seq_len,
        )

        #128*100*10
        logits = self.emb_linear(H_o)
        H = self.emb_sigmoid(logits)

        return H
    
class RecoveryNetwork(nn.Module):
    """The recovery network (decoder) for TimeGAN
    """
    def __init__(self,hidden_dim,feature_dim,num_layers,padding_value=0,max_seq_len=1000):
        super(RecoveryNetwork, self).__init__()
        self.hidden_dim = hidden_dim
        self.feature_dim = feature_dim
        self.num_layers = num_layers
        self.padding_value = padding_value
        self.max_seq_len = max_seq_len

        #Recovery Architecture
        self.rec_rnn = nn.GRU(
            input_size=self.hidden_dim,
            hidden_size=self.hidden_dim,
            num_layers=self.num_layers,
            batch_first=True,
        )

        self.rec_linear = nn.Linear(self.hidden_dim, self.feature_dim)

        # Init weights
        # Default weights of TensorFlow is Xavier Uniform for W and 1 or 0 for b
        # Reference: 
        # - https://www.tensorflow.org/api_docs/python/tf/compat/v1/get_variable
        # - https://github.com/tensorflow/tensorflow/blob/v2.3.1/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py#L484-L614

        with torch.no_grad():
            for name,param in self.rec_rnn.named_parameters():
                if 'weight_ih' in name:
                    nn.init.xavier_uniform_(param.data)
                elif 'weight_hh' in name:
                    nn.init.xavier_uniform_(param.data)
                elif 'bias_ih' in name:
                    param.data.fill_(1)
                elif 'bias_hh' in name:
                    param.data.fill_(0)
            for name,param in self.rec_linear.named_parameters():
                if 'weight' in name:
                    nn.init.xavier_uniform_(param)
                elif 'bias' in name:
                    param.data.fill_(0)
        
    def forward(self,H,T):
        """ Forward pass of the recovery features from latent space to original space.
        Args:
            H: latent representation (B x S x E)
            T: input temporal information (B)
        Returns:
            X_tilde: recovered features (B x S x F)
        """
        #Dynamic RNN input for ignoring paddings
        H_pack = nn.utils.rnn.pack_padded_sequence(
            input = H,
            lengths=T,
            batch_first=True,
            enforce_sorted=False,
        )
        #128 x 100 x 10
        H_o,H_t = self.rec_rnn(H_pack)
        #pad RNN output back to sequence length
        H_o,T = nn.utils.rnn.pad_packed_sequence(
            sequence=H_o,
            batch_first=True,
            padding_value=self.padding_value,
            total_length=self.max_seq_len,
        )
        #128 x 100 x 71
        X_tilde = self.rec_linear(H_o)
        return X_tilde

class SupervisorNetwork(nn.Module):
        """The supervisor network for TimeGAN
        """
        def __init__(self,hidden_dim,num_layers,padding_value=0,max_seq_len=1000):
            super(SupervisorNetwork,self).__init__()
            self.hidden_dim =hidden_dim
            self.num_layers = num_layers
            self.padding_value = padding_value
            self.max_seq_len = max_seq_len

            #supervisor architecture
            self.sup_rnn = nn.GRU(
                input_size=self.hidden_dim,
                hidden_size=self.hidden_dim,
                num_layers=self.num_layers-1,
                batch_first=True,
            )
            self.sup_linear = nn.Linear(self.hidden_dim,self.hidden_dim)
            self.sup_sigmoid = nn.Sigmoid()
             # Init weights
            # Default weights of TensorFlow is Xavier Uniform for W and 1 or 0 for b
            # Reference: 
            # - https://www.tensorflow.org/api_docs/python/tf/compat/v1/get_variable
            # - https://github.com/tensorflow/tensorflow/blob/v2.3.1/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py#L484-L614
            with torch.no_grad():
                for name, param in self.sup_rnn.named_parameters():
                    if 'weight_ih' in name:
                        torch.nn.init.xavier_uniform_(param.data)
                    elif 'weight_hh' in name:
                        torch.nn.init.xavier_uniform_(param.data)
                    elif 'bias_ih' in name:
                        param.data.fill_(1)
                    elif 'bias_hh' in name:
                        param.data.fill_(0)
                for name, param in self.sup_linear.named_parameters():
                    if 'weight' in name:
                        torch.nn.init.xavier_uniform_(param)
                    elif 'bias' in name:
                        param.data.fill_(0)
        def forward(self,H,T):
            """Forward pass for the supervisor for predicting next step
            Args:
                H: latent representation (B x S x E)
                T: input temporal information (B)
            Returns:
                H_hat: predicted next step data (B x S x E)
            """

            #Dynamic RNN input for ignoring paddings
            H_pack = nn.utils.rnn.pack_padded_sequence(
                input = H,
                lengths=T,
                batch_first=True,
                enforce_sorted=False,
            )

            H_o,H_t = self.sup_rnn(H_pack)
            #pad RNN output back to sequence length
            H_o,T = nn.utils.rnn.pad_packed_sequence(
                sequence=H_o,
                batch_first=True,
                padding_value=self.padding_value,
                total_length=self.max_seq_len,
            )
            logits = self.sup_linear(H_o)
            H_hat = self.sup_sigmoid(logits)
            return H_hat

class GeneratorNetwork(nn.Module):
    """The generator network for TimeGAN
    """
    def __init__(self,Z_dim,hidden_dim,num_layers,padding_value=0,max_seq_len=1000):
        super(GeneratorNetwork,self).__init__()
        self.Z_dim = Z_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.padding_value = padding_value
        self.max_seq_len = max_seq_len

        #Generator Architecture
        self.gen_rnn = nn.GRU(
            input_size=self.Z_dim,
            hidden_size=self.hidden_dim,
            num_layers=self.num_layers,
            batch_first=True,
        )
        self.gen_linear = nn.Linear(self.hidden_dim,self.hidden_dim)
        self.gen_sigmoid = nn.Sigmoid()
                # Init weights
        # Default weights of TensorFlow is Xavier Uniform for W and 1 or 0 for b
        # Reference: 
        # - https://www.tensorflow.org/api_docs/python/tf/compat/v1/get_variable
        # - https://github.com/tensorflow/tensorflow/blob/v2.3.1/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py#L484-L614
        with torch.no_grad():
            for name, param in self.gen_rnn.named_parameters():
                if 'weight_ih' in name:
                    torch.nn.init.xavier_uniform_(param.data)
                elif 'weight_hh' in name:
                    torch.nn.init.xavier_uniform_(param.data)
                elif 'bias_ih' in name:
                    param.data.fill_(1)
                elif 'bias_hh' in name:
                    param.data.fill_(0)
            for name, param in self.gen_linear.named_parameters():
                if 'weight' in name:
                    torch.nn.init.xavier_uniform_(param)
                elif 'bias' in name:
                    param.data.fill_(0)
        
    def forward(self,Z,T):
        """ Takes in random noise (features) and generates synthetic features within the last latent space
        Args:
            Z: input random noise (B x S x Z)
            T: input temporal information (B)
        Returns:
            H: embeddings (B x S x E)
        """
        #Dynamic RNN input for ignoring paddings
        Z_pack = nn.utils.rnn.pack_padded_sequence(
            input = Z,
            lengths=T,
            batch_first=True,
            enforce_sorted=False,
        )

        # 128*100*71
        H_o,H_t = self.gen_rnn(Z_pack)

        #pad RNN output back to sequence length

        H_o,T = nn.utils.rnn.pad_packed_sequence(
            sequence=H_o,
            batch_first=True,
            padding_value=self.padding_value,
            total_length=self.max_seq_len,
        )

        #128*100*10
        logits = self.gen_linear(H_o)
        H = self.gen_sigmoid(logits)

        return H

class DiscriminatorNetwork(nn.Module):
    """The discriminator network for TimeGAN
    """
    def __init__(self,hidden_dim,num_layers,padding_value=0,max_seq_len=1000   ):
        super(DiscriminatorNetwork,self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.padding_value = padding_value
        self.max_seq_len = max_seq_len

        #Discriminator Architecture
        self.dis_rnn = nn.GRU(
            input_size=self.hidden_dim,
            hidden_size=self.hidden_dim,
            num_layers=self.num_layers,
            batch_first=True,
        )
        self.dis_linear = nn.Linear(self.hidden_dim,1)

        # Init weights
        # Default weights of TensorFlow is Xavier Uniform for W and 1 or 0 for b
        # Reference: 
        # - https://www.tensorflow.org/api_docs/python/tf/compat/v1/get_variable
        # - https://github.com/tensorflow/tensorflow/blob/v2.3.1/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py#L484-L614
        with torch.no_grad():
            for name, param in self.dis_rnn.named_parameters():
                if 'weight_ih' in name:
                    torch.nn.init.xavier_uniform_(param.data)
                elif 'weight_hh' in name:
                    torch.nn.init.xavier_uniform_(param.data)
                elif 'bias_ih' in name:
                    param.data.fill_(1)
                elif 'bias_hh' in name:
                    param.data.fill_(0)
            for name, param in self.dis_linear.named_parameters():
                if 'weight' in name:
                    torch.nn.init.xavier_uniform_(param)
                elif 'bias' in name:
                    param.data.fill_(0)
    
    def forward(self, H, T):
        """ Forward pass for predicting if the data is real or synthetic
        
        Args:
            H: latent representation (B x S x E)
            T: input temporal information (B)
        Returns:
        logits: prediction logits(B x S x 1)
        """
        # dynamic RNN input for ignoring paddings
        H_pack = nn.utils.rnn.pack_padded_sequence(
            input = H,
            lengths=T,
            batch_first=True,
            enforce_sorted=False,
        )

        # 128*100*10
        H_o,H_t = self.dis_rnn(H_pack)

        # pad RNN output back to sequence length
        H_o,T = nn.utils.rnn.pad_packed_sequence(
            sequence=H_o,
            batch_first=True,
            padding_value=self.padding_value,
            total_length=self.max_seq_len,
        )

        logits = self.dis_linear(H_o).squeeze(-1)
        return logits

class TimeGAN(nn.Module):
    """ Implementation of TimeGan (Yoon et al., 2019) using PyTorch
    
    Reference:
        - Yoon, J., Jarret, D., van der Schaar, M. (2019). Time-series Generative Adversarial Networks. (https://papers.nips.cc/paper/2019/hash/c9efe5f26cd17ba6216bbe2a7d26d490-Abstract.html)
        - https://github.com/jsyoon0823/TimeGAN
    """
    def __init__(self,device,feature_dim,Z_dim,hidden_dim,max_seq_len,batch_size,num_layers,padding_value):
        super(TimeGAN,self).__init__()
        self.device =device
        self.feature_dim = feature_dim
        self.Z_dim = Z_dim
        self.hidden_dim = hidden_dim
        self.max_seq_len = max_seq_len
        self.batch_size = batch_size

        self.embedder = EmbeddingNetwork(feature_dim=feature_dim,hidden_dim=hidden_dim,num_layers=num_layers,padding_value=padding_value,max_seq_len=max_seq_len)
        self.recovery = RecoveryNetwork(feature_dim=feature_dim,hidden_dim=hidden_dim,num_layers=num_layers,padding_value=padding_value,max_seq_len=max_seq_len)
        self.generator = GeneratorNetwork(Z_dim=Z_dim,feature_dim=feature_dim,hidden_dim=hidden_dim,num_layers=num_layers,padding_value=padding_value,max_seq_len=max_seq_len)
        self.discriminator = DiscriminatorNetwork(feature_dim=feature_dim,hidden_dim=hidden_dim,num_layers=num_layers,padding_value=padding_value,max_seq_len=max_seq_len)
        self.supervisor = SupervisorNetwork(feature_dim=feature_dim,hidden_dim=hidden_dim,num_layers=num_layers,padding_value=padding_value,max_seq_len=max_seq_len)

    def _recovery_forward(self, X, T):
        """ The embedding network forward pass and the embedder network loss
        Args:
            X: input features
            T: input temporal information
        Returns:
            E_loss: the reconstruction loss
            X_tilde: the reconstructed features
        """

        # FOrward pass
        H = self.embedder(X,T)
        X_tilde = self.recovery(H,T)

        #for Joint training
        H_hat_supervise = self.supervisor(H,T)
        G_loss_S = F.mse_loss(
            H_hat_supervise[:,:-1,:],
            H[:,1:,:],
        ) #Teacher forcing next output

        #Reconstruction loss
        E_loss_T0 = F.mse_loss(X_tilde,X)
        E_loss0 = 10*torch.sqrt(E_loss_T0)
        E_loss = E_loss0 + 0.1*G_loss_S
        return E_loss, E_loss0,E_loss_T0
    def _supervisor_forward(self, X, T):
        """ The supervisor training forward pass
        Args:
            X: original input features
            T: input temporal information
        Returns:
            S_loss: the supervisor's loss
        """
        #supervisor forward pass
        H = self.embedder(X,T)
        H_hat_supervise = self.supervisor(H,T)

        #supervised loss
        S_loss = F.mse_loss(
            H_hat_supervise[:,:-1,:],
            H[:,1:,:],
        ) #Teacher forcing next output
        return S_loss
    def _discriminator_forward(self, X, T, Z, gamma=1):
        """ The discriminator forward pass and adversarial loss
        Args:
            X: input features
            T: input temporal information
            Z: input noise
            gamma: the weight for the adversarial loss
        Returns:
            D_loss: adversarial loss
        """
        #Real
        H = self.embedder(X, T).detach()

        #generator
        E_hat = self.generator(Z,T).detach()
        H_hat = self.supervisor(E_hat,T).detach()
        
        #forward pass
        Y_real = self.discriminator(H,T)        #Encode original data
        Y_fake = self.discriminator(H_hat,T)    #Output of generator + supervisor
        Y_fake_e = self.discriminator(E_hat,T)  #Output of generator

        D_loss_real = F.binary_cross_entropy_with_logits(Y_real, torch.ones_like(Y_real))
        D_loss_fake = F.binary_cross_entropy_with_logits(Y_fake, torch.zeros_like(Y_fake))
        D_loss_fake_e = F.binary_cross_entropy_with_logits(Y_fake_e, torch.zeros_like(Y_fake_e))

        D_loss = D_loss_real + D_loss_fake + gamma * D_loss_fake_e

        return D_loss
    
    def _generator_forward(self, X, T, Z, gamma=1):
        """ The generator forward pass
        Args:
            X: original input features
            T: input temporal information
            Z: input noise for the generator
            gamma: the weight for the adversarial loss
        Returns:
            G_loss: the generator loss
        """
        #supervisor forward pass
        H = self.embedder(X,T)
        H_hat_supervise = self.supervisor(H,T)

        #generator forward pass
        E_hat = self.generator(Z,T)
        H_hat = self.supervisor(E_hat,T)

        #synthetic data generated
        X_hat = self.recovery(H_hat,T)

        #generator loss
        #Adversarial loss
        Y_fake = self.discriminator(H_hat,T)        #Output of supervisor
        Y_fake_e = self.discriminator(E_hat,T)      #Output of generator

        G_loss_U = F.binary_cross_entropy_with_logits(Y_fake, torch.ones_like(Y_fake))
        G_loss_U_e = F.binary_cross_entropy_with_logits(Y_fake_e, torch.ones_like(Y_fake_e))

        #Supervised loss
        G_loss_S = F.mse_loss(
            H_hat_supervise[:,:-1,:],
            H[:,1:,:],
        ) #Teacher forcing next output

        #Two moments losses
        G_loss_V1 = torch.mean(
            torch.abs(torch.sqrt(X_hat.var(dim=0,unbiased=False)+1e-6) - torch.sqrt(X.var(dim=0,unbiased=False)+1e-6))
        )
        G_loss_V2 = torch.mean(torch.abs((X_hat.mean(dim=0)) - (X.mean(dim=0))))
        G_loss_V = G_loss_V1 + G_loss_V2
        
        #sum of losses
        G_loss = G_loss_U + gamma * G_loss_U_e + 100 * torch.sqrt(G_loss_S) + 100 * G_loss_V
    
        return G_loss
    
    def _inference(self, Z,T):
        """ Inference for generating synthetic data
        Args:
            Z: input noise
            T: temporal information
        Returns:
            X_hat: the generated data
        """

        #generator forward pass
        E_hat = self.generator(Z,T)
        H_hat = self.supervisor(E_hat,T)

        #synthetic data generated
        X_hat = self.recovery(H_hat,T)
        return X_hat

    def forward(self,X,T,Z, obj, gamma=1):
        """
        Args:
            X: input features (B,H,F)
            T: The temporal information (B)
            Z: the sampled noise (B,H,Z)
            obj: the network to be trained ('autoencoder','supervisor','generator','discriminator')
            gamma: loss hyperparameter
        Returns:
            loss: loss for the forward pass
            X_hat: the generated data
        """

        #Move variables to device
        if obj !='inference':
            if X is None:
                raise ValueError('X cannot be empty')
            
            X = torch.FloatTensor(X)
            X = X.to(self.device)

        if Z is not None:
            Z = torch.FloatTensor(Z)
            Z = Z.to(self.device)
        
        if obj == 'autoencoder':
            #embedder and recovery forward
            loss = self._recovery_forward(X,T)
        elif obj == 'supervisor':
            loss = self._supervisor_forward(X,T)
        elif obj == 'generator':
            if Z is None:
                raise ValueError('Z cannot be empty')
            loss = self._generator_forward(X,T,Z,gamma)
        elif obj == 'discriminator':
            if Z is None:
                raise ValueError('Z cannot be empty')
            loss = self._discriminator_forward(X,T,Z,gamma)
            return loss
        elif obj == 'inference':
            X_hat = self._inference(Z,T)
            X_hat = X_hat.cpu.detach()

            return X_hat
        else:
            raise ValueError('obj must be autoencoder, supervisor, generator or discriminator')
        return loss
        

## Trainers

In [22]:
def embedding_trainer(
        model: torch.nn.Module,
        dataloader: torch.utils.data.DataLoader,
        e_opt: torch.optim.Optimizer,
        r_opt: torch.optim.Optimizer,
        emb_epochs: int,
):
    """
    Training loop for embedding and recovery functions.
    Args:
        model (torch.nn.Module): The model to train
        dataloader (torch.utils.data.DataLoader): The dataloader to use
        e_opt (torch.optim.Optimizer): The optimizer for the embedding function
        r_opt (torch.optim.Optimizer): The optimizer for the recovery function
        args (Dict): The model/training configuration
        writer (Union[torch.utils.tensorboard.SummaryWriter, type(None)], optional): The tensorboard writer to use. Defaults to None.
    """
    logger = trange(emb_epochs, desc =f"Epoch:0, Loss:0")
    for epoch in logger:
        for X_mb,T_mb in dataloader:

            #reset gradients
            model.zero_grad()

            #forward pass
            _,E_loss0,E_loss_T0 = model(X=X_mb,T=T_mb,Z=None,obj="autoencoder")
            loss = np.sqrt(E_loss_T0.item())

            #backward pass
            E_loss0.backward()

            #update weights
            e_opt.step()
            r_opt.step()

        # Log loss for final batch of each epochs
        logger.set_description(f"Epoch:{epoch}, Loss:{loss:.4f}")
        """if writer:
            writer.add_scalar("Embedding/Loss:",loss,epoch)
            writer.flush()"""

def supervisor_trainer(
    model: torch.nn.Module, 
    dataloader: torch.utils.data.DataLoader, 
    s_opt: torch.optim.Optimizer, 
    g_opt: torch.optim.Optimizer,
    sup_epochs: int,
):
    """
    The training loop for the supervisor function
    Args:
        model (torch.nn.Module): The model to train
        dataloader (torch.utils.data.DataLoader): The dataloader to use
        s_opt (torch.optim.Optimizer): The optimizer for the supervisor function
        g_opt (torch.optim.Optimizer): The optimizer for the generator function
        args (Dict): The model/training configuration
        writer (Union[torch.utils.tensorboard.SummaryWriter, type(None)], optional): The tensorboard writer to use. Defaults to None.
    """
    logger = trange(sup_epochs, desc=f"Epoch: 0, Loss: 0")
    for epoch in logger:
        for X_mb, T_mb in dataloader:
            # Reset gradients
            model.zero_grad()

            # Forward Pass
            S_loss = model(X=X_mb, T=T_mb, Z=None, obj="supervisor")

            # Backward Pass
            S_loss.backward()
            loss = np.sqrt(S_loss.item())

            # Update model parameters
            s_opt.step()

        # Log loss for final batch of each epoch (29 iters)
        logger.set_description(f"Epoch: {epoch}, Loss: {loss:.4f}")
        """if writer:
            writer.add_scalar(
                "Supervisor/Loss:",loss,epoch)
            writer.flush()"""

In [23]:
def joint_trainer(
    model: torch.nn.Module, 
    dataloader: torch.utils.data.DataLoader, 
    e_opt: torch.optim.Optimizer, 
    r_opt: torch.optim.Optimizer, 
    s_opt: torch.optim.Optimizer, 
    g_opt: torch.optim.Optimizer, 
    d_opt: torch.optim.Optimizer,
    sup_epochs: int,
    batch_size: int,
    max_seq_len: int,
    Z_dim: int,
    dis_thresh: float,
    ):
    """
    The training loop for training the model altogether
    Args:
        model (torch.nn.Module): The model to train
        dataloader (torch.utils.data.DataLoader): The dataloader to use
        e_opt (torch.optim.Optimizer): The optimizer for the embedding function
        r_opt (torch.optim.Optimizer): The optimizer for the recovery function
        s_opt (torch.optim.Optimizer): The optimizer for the supervisor function
        g_opt (torch.optim.Optimizer): The optimizer for the generator function
        d_opt (torch.optim.Optimizer): The optimizer for the discriminator function
        args (Dict): The model/training configuration
        writer (Union[torch.utils.tensorboard.SummaryWriter, type(None)], optional): The tensorboard writer to use. Defaults to None.
    """
    logger = trange(
        sup_epochs, 
        desc=f"Epoch: 0, E_loss: 0, G_loss: 0, D_loss: 0"
    )
    for epoch in logger:
        for X_mb, T_mb in dataloader:
            ## Generator Training
            for _ in range(2):
                # Random Generator
                Z_mb = torch.rand((batch_size, max_seq_len, Z_dim))

                # Forward Pass (Generator)
                model.zero_grad()
                G_loss = model(X=X_mb, T=T_mb, Z=Z_mb, obj="generator")
                G_loss.backward()
                G_loss = np.sqrt(G_loss.item())

                # Update model parameters
                g_opt.step()
                s_opt.step()

                # Forward Pass (Embedding)
                model.zero_grad()
                E_loss, _, E_loss_T0 = model(X=X_mb, T=T_mb, Z=Z_mb, obj="autoencoder")
                E_loss.backward()
                E_loss = np.sqrt(E_loss.item())
                
                # Update model parameters
                e_opt.step()
                r_opt.step()

            # Random Generator
            Z_mb = torch.rand((batch_size, max_seq_len, Z_dim))

            ## Discriminator Training
            model.zero_grad()
            # Forward Pass
            D_loss = model(X=X_mb, T=T_mb, Z=Z_mb, obj="discriminator")

            # Check Discriminator loss
            if D_loss > dis_thresh:
                # Backward Pass
                D_loss.backward()

                # Update model parameters
                d_opt.step()
            D_loss = D_loss.item()

        logger.set_description(
            f"Epoch: {epoch}, E: {E_loss:.4f}, G: {G_loss:.4f}, D: {D_loss:.4f}"
        )
        """if writer:
            writer.add_scalar(
                'Joint/Embedding_Loss:',E_loss,epoch)
            writer.add_scalar(
                'Joint/Generator_Loss:',G_loss,epoch)
            writer.add_scalar('Joint/Discriminator_Loss:',D_loss,epoch)
            writer.flush()"""

def timegan_trainer(model,train_data,train_time,batch_size,lr,emb_epochs,sup_epochs,max_seq_length,Z_dim,dis_thresh,device,model_path):
    """
    The trainign procedure for TimeGAN.
    Args:
        model (torch.nn.module): The model that generates synthetic data
        loaded_data(pandas.DataFrame): The data to train on, including data and time
        args (Dict): The model/training configuration
    Returns:
        generated_data (np.array): The synthetic data generated by the model
    """
    dataset = TimeGAN_Dataset(data=train_data,time=train_time)
    dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False)
    model.to(device)

    #initialize optimizers
    e_opt = torch.optim.Adam(model.embedder.parameters(), lr=lr)
    r_opt = torch.optim.Adam(model.recovery.parameters(), lr=lr)
    s_opt = torch.optim.Adam(model.supervisor.parameters(), lr=lr)
    g_opt = torch.optim.Adam(model.generator.parameters(), lr=lr)
    d_opt = torch.optim.Adam(model.discriminator.parameters(), lr=lr)

    #initialize tensorboard writer
    #writer = SummaryWriter(os.path.join(f"tensorboard/{args.exp}"))

    print("\nStart Embedding Network Training")
    embedding_trainer(model=model, dataloader=dataloader, e_opt=e_opt, r_opt=r_opt, emb_epochs=emb_epochs)

    print("\nStart Training with Supervised Loss Only")
    supervisor_trainer(model=model, dataloader=dataloader, s_opt=s_opt,g_opt=g_opt, sup_epochs=sup_epochs)

    print("\nStart Joint Training")
    joint_trainer(model=model, dataloader=dataloader, e_opt=e_opt, r_opt=r_opt, s_opt=s_opt, g_opt=g_opt, d_opt=d_opt, sup_epochs=sup_epochs, batch_size=batch_size, max_seq_len=max_seq_length, Z_dim=Z_dim, dis_thresh=dis_thresh)


    #save model,args, and hyperparameters
    #torch.save(args,f"{args.model_path}/args.pickle")
    torch.save(model.state_dict(),f"{model_path}/model.pt")
    print(f"Model saved to {model_path}")

def timegan_generator(model,T,model_path,batch_size, max_seq_len, Z_dim ,device):
    """
    The interference procedure for TimeGAN.
    Args:
        model (torch.nn.module): The model that generates synthetic data
        T (List[int]): The time to generate data for
        args (Dict): The model/training configuration
    returns:
        generated_data (np.array): The synthetic data generated by the model
    """
    #load model
    if not os.path.exists(model_path):
        raise ValueError(f"Model not found at {model_path}")
    model.load_state_dict(torch.load(f"{model_path}/model.pt"))
    print("\nStart Generating Synthetic Data")
    #Initialize model to evaluation mode and run without gradients
    model.to(device)
    model.eval()
    with torch.no_grad():
        # Random Generator
        Z = torch.rand((batch_size, max_seq_len, Z_dim))
        # Forward Pass (Generator)
        generated_data = model(X=None, T=T, Z=Z, obj="inference")
    return generated_data.numpy()

# Data handling

## Data loading

In [24]:
def load_data(data_limit=None,save_dataset=None):
    """
    data_limit=None,save_dataset=None
    Load and preprocess real life datasets.
    
    Args:
        data_limit (int): The number of data points to load. If None, all data points are loaded. Default: None. Used for testing.
        save_dataset (bool): If 'Full', the dataset is saved to a csv file. If it's 'limited', than save the limited dataset if data_limit is not None. Default: None.


    Returns:
        dataset (pandas.DataFrame): The dataset.
    
    """

    main_dataset = pd.DataFrame()

    
    Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
    filenames = askopenfilenames() # show an "Open" dialog box and return the path to the selected file
    print(filenames)
    for filename in filenames:
        
        if filename.endswith('.mat'):
            #output df format: [id,value_array]
            df = load_mat_as_df(filename)
            print(df)
        
        elif filename.endswith('.csv'):

            #use create_dataset_csv.py to create a csv file
            if filename.find('dataset') != -1:
                df = pd.read_csv(filename,sep=';',index_col=0)

            """ CSV format:
            ID|time|Sleeping stage|length|additional_info

            """
            pass

        elif filename.endswith('.xml'):
            ## TODO: add xml support
            #df =
            pass
        else:
            print("Unsupported file format, skipping file:",filename,".")
            pass
    
        main_dataset.append(df)

    #Cut df to data_limit size for testing purposes
    if data_limit is not None:
        if save_dataset == 'Full':
            #save dataset to a csv file
            main_dataset.to_csv('Full_dataset.csv',sep=';')
        
        elif save_dataset == 'Limited':
            main_dataset = main_dataset[:data_limit]
            #save dataset to a csv file
            main_dataset.to_csv('limited_dataset.csv',sep=';')
        elif save_dataset == 'None':
            main_dataset = main_dataset[:data_limit]
            pass
        else:
            raise ValueError("Invalid save_dataset value, valid values are 'Full','Limited','None'.")

    elif data_limit is None:
        if save_dataset == 'Full':
            #save dataset to a csv file
            main_dataset.to_csv('full_dataset.csv',sep=';')
        elif save_dataset == 'Limited':
            print("Warning: data_limit is None, dataset is not limited, saving full dataset.")
            main_dataset.to_csv('full_dataset.csv',sep=';')
        elif save_dataset == 'None':
            pass
        else:
            raise ValueError("Invalid save_dataset value, valid values are 'Full','Limited','None'.")

    
    return main_dataset #dataset as df

def load_mat_as_df(mat_file_path, var_name):
    mat = sio.loadmat(mat_file_path,simplify_cells=True)

    if var_name not in list(mat.keys()):
        var_name = get_variable_name(mat)   
        

    return pd.DataFrame(mat[var_name])

def get_variable_name(loaded_mat):

    
    root = tk.Tk()
    root.title('.mat variable selector')
    tk.Label(root, text="Choose a variable:").pack()
    choices = list(loaded_mat.keys())

    variable = tk.StringVar(root)
    variable.set(choices[0]) # default value
    w = tk.Combobox(root, textvariable=variable,values=choices)

    w.pack()
    def ok():
        print ("value is:" + variable.get())
        root.destroy()
    def cancel():
        root.destroy()
        raise InterruptedError('User cancelled, invalid variable name')

    button1 = tk.Button(root, text="OK", command=ok)
    button2 = tk.Button(root, text="Cancel", command=cancel)
    button1.pack()
    button2.pack()
    root.mainloop()
    
    return variable.get()

## Data preprocess

In [25]:
def preprocess_data(padding_value,data_limit=None,save_dataset=None,norm_enable=False):
    """
    padding_value: int = -1.0,
    data_limit: int = None"""
    # Load and preprocess data
    # 
    # 1. Load data from files (csv,mat,xml)
    # 2. Preprocess data:
    # 2.1. Remove outliers
    # 2.2. Extract sequence length and time
    # 2.3. Resample data
    # 2.4. Normalize data
    # 2.5. Padding 
    #  
    # Args:
    #     data_limit (int): The number of lines to load from the data file
    #     padding_value (int): The value used for padding
    #     
    # 
    # Returns:
    #     prep_data (pandas.DataFrame): The processed data

    #######################################
    # 1. Load data from files (csv,mat,xml)
    #######################################

    loaded_data = load_data(data_limit=data_limit,save_dataset=save_dataset)
    """
    loaded data =       time_data   , data  , length
    (pandas.DataFrame), (np.array)  ,(list) ,(int)
    
    ()
    """
    #######################################
    # 2. Preprocess data:
    #######################################
    # 2.1. Remove outliers
    #######################################
    """
    Remove row's with unacceptable sleep stages values
    """
    sleep_stages = np.array([1,2,3,4,5])
    loaded_data[loaded_data['data'].apply(lambda x: all(elem in sleep_stages for elem in x))]

    #######################################
    # 2.2. Extract sequence length and time
    #######################################
    """
    Extract sequence length of all lines and time of each line
    """
    loaded_data['length'] = loaded_data['data'].apply(lambda x: len(x))

    
    #######################################
    # 2.4. Normalize data
    #######################################
    """
    Normalize data to [0,1] using MinMaxScaler algorithm
    """

    

    if norm_enable == True:
        loaded_data['data']=MinMaxNormalizer(loaded_data['data'])
    

    #######################################
    # 2.5. Padding
    #######################################
    """
    Padding data to given length
    """
   
    # Question: Current padding value is 0, is it ok? Do we need it or just resample?
    data_info = {
        'length' : len(loaded_data),
        'max_length' : max(loaded_data['length']),
        'paddding_value' : padding_value,

    }
    
    loaded_data['data'] = loaded_data['data'].apply(lambda x: np.transpose(x))
    prep_data = pd.DataFrame(columns=['time','data'])
    for i in tqdm(range(data_info.length)):
        #create empty array with padding value
        tmp_array = np.empty([data_info.max_length,1])
        tmp_array.fill(padding_value)
        #fill array with data
        tmp_array[:loaded_data['data'][i].shape[0],:loaded_data['data'][i].shape[1]] = loaded_data['data'][i]
        #append to prep_data
        prep_data.append(tmp_array)

    return prep_data.to_numpy(),loaded_data['time_data'].to_numpy(),data_info
    

    
    
    


def MinMaxNormalizer(data,min_value=1,max_value=5):
    numerator = data-min_value
    denominator = max_value-min_value
    norm_data = numerator/denominator
    return norm_data

# Main

In [26]:
#######################################
# Experiment Arguments
#######################################

# select device:
device = "cuda"

#set random seed
seed = 0

#experiment name
exp_name ="test"

#normalization enable
norm_enable = False

# padding value
padding_value = -1.0

# Train
is_train = True

# dataset save:
# Full: save full dataset
# Limited: save limited dataset
# None: don't save dataset
save_dataset = "None"

# save arguments to neptune
run["Initialization/Arguments/ExperimentArgs/device"] = device
run["Initialization/Arguments/ExperimentArgs/seed"] = seed
run["Initialization/Arguments/ExperimentArgs/exp_name"] = exp_name
run["Initialization/Arguments/ExperimentArgs/norm_enable"] = norm_enable
run["Initialization/Arguments/ExperimentArgs/padding_value"] = padding_value
run["Initialization/Arguments/ExperimentArgs/is_train"] = is_train
run["Initialization/Arguments/ExperimentArgs/save_dataset"] = save_dataset


#######################################
# Data Arguments
#######################################

#data limit for testing
# None: use full dataset
# int: use limited dataset
data_limit = 2

#train test split rate
train_rate = 0.6
# save arguments to neptune
run["Initialization/Arguments/DataArgs/data_limit"] = data_limit
run["Initialization/Arguments/DataArgs/train_rate"] = train_rate

#######################################
# Model Arguments
#######################################

# embedding model epochs
emb_epochs = 600

# GAN model epochs
gan_epochs = 600

# supervised model epochs
sup_epochs = 600

# batch size
batch_size = 64

# hidden dimension of RNN
hidden_dim = 20

# number of layers in RNN
num_layers = 3

# discriminator threshold
dis_thresh = 0.15

#learning rate
lr = 1e-3


# save arguments to neptune
run["Initialization/Arguments/ModelArgs/embedding_epochs"] = emb_epochs
run["Initialization/Arguments/ModelArgs/gan_epochs"] = gan_epochs
run["Initialization/Arguments/ModelArgs/supervised_epochs"] = sup_epochs
run["Initialization/Arguments/ModelArgs/batch_size"] = batch_size
run["Initialization/Arguments/ModelArgs/hidden_dim"] = hidden_dim
run["Initialization/Arguments/ModelArgs/num_layers"] = num_layers
run["Initialization/Arguments/ModelArgs/discriminator_threshold"] = dis_thresh
run["Initialization/Arguments/ModelArgs/learning_rate"] = lr



######################################
# Initialize output directories
######################################
## runtime directory
code_dir = os.path.abspath(".")
if not os.path.exists(code_dir):
    raise ValueError(f"Code directory not found at {code_dir}")

## Data directory
data_path = os.path.abspath("./Data")
if not os.path.exists(data_path):
    raise ValueError(f"Data directory not found at {data_path}")
data_dir = os.path.dirname(data_path)
data__file_name = os.path.basename(data_path)
## Output directory
model_path = os.path.abspath(f"./Output/{exp_name}/")
out_dir = os.path.abspath(model_path)
if not os.path.exists(out_dir):
    os.makedirs(out_dir,exist_ok=True)

print(f"Code directory:\t\t\t{code_dir}")
print(f"Data directory:\t\t\t{data_path}")
print(f"Output directory:\t\t{out_dir}")

 ######################################
# Initialize random seed and CUDA
######################################
os.environ["PYTHONHASHSEED"] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if device == "cuda" and torch.cuda.is_available():
    print("Using CUDA\n")
    device = torch.device("cuda:0")
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    print("Using CPU\n")
    device = torch.device("cpu")

######################################
# Load and preprocess data for model
######################################
X,T,loaded_data_info = preprocess_data(padding_value=padding_value,data_limit=data_limit,save_dataset=save_dataset,norm_enable=norm_enable)
print(f"Processed Data: {X.shape} (Idx x Max_Sequence_Length x Features(=1))")
print(f"Original data preview:\n{X[:2, :10, :2]}\n")
feature_dim = X.shape[-1]
Z_dim = X.shape[-1]
max_seq_len = loaded_data_info.max_length
 # Train-Test Split data and time
 # TODO: Same people should be in the same pool at train test split
 # Make the split on the subject ID's, so also have to att ID to the loaded data
train_data, test_data, train_time, test_time = train_test_split(
    X, T, test_size=train_rate, random_state=seed
)
#########################
# Initialize and Run model
#########################
 # Log start time
start = time.time()
model = TimeGAN(feature_dim=feature_dim,  hidden_dim=hidden_dim, num_layers=num_layers, max_seq_len=max_seq_len,padding=padding_value)
if is_train == True:
    timegan_trainer(model,train_data,train_time,batch_size=batch_size,lr=lr,emb_epochs=emb_epochs,sup_epochs=sup_epochs,max_seq_length=max_seq_len,Z_dim=Z_dim,dis_thresh=dis_thresh,device=device,model_path=model_path)
generated_data = timegan_generator(model, train_time, model_path=model_path,batch_size=batch_size,max_seq_length=max_seq_len,Z_dim=Z_dim,device=device)
generated_time = train_time
# Log end time
end = time.time()

print(f"Generated data preview:\n{generated_data[:2, -10:, :2]}\n")
print(f"Model Runtime: {(end - start)/60} mins\n")
# Save splitted data and generated data
with open(f"{model_path}/train_data.pickle", "wb") as fb:
    pickle.dump(train_data, fb)
with open(f"{model_path}/train_time.pickle", "wb") as fb:
    pickle.dump(train_time, fb)
with open(f"{model_path}/test_data.pickle", "wb") as fb:
    pickle.dump(test_data, fb)
with open(f"{model_path}/test_time.pickle", "wb") as fb:
    pickle.dump(test_time, fb)
with open(f"{model_path}/fake_data.pickle", "wb") as fb:
    pickle.dump(generated_data, fb)
with open(f"{model_path}/fake_time.pickle", "wb") as fb:
    pickle.dump(generated_time, fb)

# Stop Neptune experiment
run.stop()

Code directory:			c:\Users\tomo9\Documents\00_School\00_Thesis\HypnoGAN
Data directory:			c:\Users\tomo9\Documents\00_School\00_Thesis\HypnoGAN\Data
Output directory:		c:\Users\tomo9\Documents\00_School\00_Thesis\HypnoGAN\Output\test
Using CUDA

Run HYPNOG-7 received stop signal. Exiting
Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/NTLAB/HypnoGAN/e/HYPNOG-7/metadata


: 

: 